In [4]:
# !pip install pandas
# !pip install geopandas
# !pip install matplotlib

In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib

In [14]:
shapefile = gpd.read_file('CUSP_IN_PROGRESS/CUSP_IN_PROGRESS.shp')
shapefile.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Status     91 non-null     object  
 1   Acquisitn  91 non-null     object  
 2   Type       91 non-null     object  
 3   geometry   91 non-null     geometry
dtypes: geometry(1), object(3)
memory usage: 3.0+ KB


In [ ]:
dado_tabulados = gpd.read_file('CUSP_IN_PROGRESS/CUSP_IN_PROGRESS.dbf')
indice_formas_geometricas = gpd.read_file('CUSP_IN_PROGRESS/CUSP_IN_PROGRESS.shx')
coordenadas = gpd.read_file('CUSP_IN_PROGRESS/CUSP_IN_PROGRESS.prj')